## Import into colmap and run reconstruction

In [1]:
feature_dir = 'featureout_loftr'
dirname = 'dirname'

from h5_to_db import add_keypoints, add_matches, COLMAPDatabase
import pycolmap
import os

def import_into_colmap(img_dir,
                       feature_dir ='.featureout',
                       database_path = 'colmap.db',
                       img_ext='.jpg'):
    db = COLMAPDatabase.connect(database_path)
    db.create_tables()
    single_camera = False
    fname_to_id = add_keypoints(db, feature_dir, img_dir, img_ext, 'simple-radial', single_camera)
    add_matches(
        db,
        feature_dir,
        fname_to_id,
    )

    db.commit()
    return

database_path = 'colmap_loftr.db'
!rm {database_path}
import_into_colmap(dirname,feature_dir=feature_dir, database_path=database_path)

 56%|█████▌    | 2542/4560 [00:01<00:00, 2178.62it/s]


In [2]:
output_path = 'colmap_rec_loftr'

In [15]:
if not os.path.isdir(output_path):
    os.makedirs(output_path)

maps = pycolmap.incremental_mapping(database_path, dirname, output_path)
maps[0].write(output_path)

In [ ]:
# dense reconstruction
pycolmap.undistort_images(output_path, output_path, dirname)
pycolmap.patch_match_stereo(output_path)  # requires compilation with CUDA
pycolmap.stereo_fusion(output_path / "dense.ply", output_path)

In [13]:
#model_dir = os.path.abspath('colmap_rec_loftr/0')
#ply_model_path = os.path.abspath('model.ply')
#!colmap model_converter --input_path "{model_dir}" --output_path "{ply_model_path}" --output_type PLY

In [15]:
#import open3d as o3d

# def visualize_ply(ply_model_path):
#     # Load the .ply file
#     pcd = o3d.io.read_point_cloud(ply_model_path)

#     # Visualize the point cloud
#     o3d.visualization.draw_geometries([pcd])

# visualize_ply('model.ply')

: 

In [9]:
# !colmap model_merger \
#     --input_path1 ./colmap_rec_loftr/0-1 \
#     --input_path2 ./colmap_rec_loftr/2 \
#     --output_path ./colmap_rec_loftr/total


I20240511 20:37:55.051832 847528 misc.cc:205] 
Reconstruction 1
----------------
I20240511 20:37:55.052505 847528 model.cc:722] Images: 97
I20240511 20:37:55.052520 847528 model.cc:723] Points: 249644
I20240511 20:37:55.348878 847528 misc.cc:205] 
Reconstruction 2
----------------
I20240511 20:37:55.348930 847528 model.cc:728] Images: 22
I20240511 20:37:55.348953 847528 model.cc:729] Points: 50033
I20240511 20:37:55.348973 847528 misc.cc:205] 
Merging reconstructions
-----------------------
I20240511 20:37:56.768738 847528 model.cc:734] => Merge succeeded
I20240511 20:37:56.768779 847528 misc.cc:205] 
Merged reconstruction
---------------------
I20240511 20:37:56.768791 847528 model.cc:736] Images: 99
I20240511 20:37:56.768801 847528 model.cc:737] Points: 255086


In [ ]:
#!colmap gui